In [6]:
import numpy as np
from sempler import LGANM

In [7]:
# Set random state
rs = np.random.RandomState(seed=42)

First, define the SCM from which to sample. For the noise variances we sample from the same interval as the original paper.

In [8]:
d = 5  # nr. of latent dims

W = np.array([[0.0, 0.0, 0.0, 0.0, 0.0],
              [0.71943922, 0.0, 0.0, 0.0, 0.67726298],
              [0.0, 0.89303215, 0.0, 0.0, 0.98534901],
              [0.84868401, 0.0, 0.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 0.0]])

variances_obs = rs.uniform(1.0, 2.0, size=d)

lganm = LGANM(W=W, means=np.zeros(d), variances=variances_obs, random_state=42)

Next, get the samples from the observational distribution. We get their mean and std_dev for later scaling

In [9]:
n = 10000  # nr. of samples per env.

obs_samples = lganm.sample(n)
means = np.mean(obs_samples, axis=0, keepdims=True)
stds = np.std(obs_samples, axis=0, keepdims=True)

obs_samples = (obs_samples - means) / stds

Z_obs = np.tile(obs_samples, (d, 1))

Now, sample from the interventional distributions. The mean and variances of the do-interventions are again sampled from the same intervals as in the original paper. We also apply the sign random sign flipping to the interventional means as the authors do.

In [10]:
means_iv = rs.uniform(1.0, 2.0, size=d)
mask = rs.binomial(n=1, p=0.5, size=means_iv.size).reshape(means_iv.shape)
means_iv = means_iv - 2 * mask * means_iv

variances_iv = rs.uniform(1.0, 2.0, size=d)

iv_samples = []
iv_targets = []
for i in range(d):
    samples = lganm.sample(n, do_interventions={i: (means_iv[i], variances_iv[i])})
    iv_samples.append(samples)
    iv_targets.append(i * np.ones(n, dtype=np.int_))

Z_iv = np.concatenate(iv_samples, axis=0)

# Rescale using the same factors as the observational data
Z_iv = (Z_iv - means) / stds

The latents are now clipped to an interval of 4 std devs around 0 in order to enable their rescaling to the correct intervals for the chamber actuators.

In [11]:
Z_obs = np.clip(Z_obs, a_min=-4, a_max=4)
Z_iv = np.clip(Z_iv, a_min=-4, a_max=4)

# Apply rescaling (that expects 4 std_devs around 0 of clipping interval)
scales_up = np.array([[255/8, 255/8, 255/8, 90/4, 90/4]])
shifts_up = np.array([[127.5, 127.5, 127.5, 0, 0]])

Z_obs_rescale = Z_obs * scales_up + shifts_up
Z_iv_rescale = Z_iv * scales_up + shifts_up